# Genome Assembly Workflow Notebook

This notebook documents the full genome assembly workflow used in the study, including preprocessing, assembly, polishing, and evaluation.  
Environment: `genome_assembly` (Python 3.10) for most steps, with Ragout running in a dedicated `ragout_env` (Python 2.7).


In [ ]:
!conda info --envs
!unicycler --version
!bwa
!samtools --version
!seqkit version
!blastn -version
!busco --version
!quast.py --version


## Step 1a. Read preprocessing

Reads were trimmed in **Geneious Prime** (following best practice guidelines).  
Here we rename read headers for consistency.


In [ ]:
!seqkit rename R1.fastq.gz -o fixed_R1.fastq
!seqkit rename R2.fastq.gz -o fixed_R2.fastq

!sed 's/^\(@[^[:space:]]*\)_1/\1/' fixed_R1.fastq > fixed_R1_clean.fastq
!sed 's/^\(@[^[:space:]]*\)_2/\1/' fixed_R2.fastq > fixed_R2_clean.fastq

!paste <(grep "^@" fixed_R1_clean.fastq | head -n 5) <(grep "^@" fixed_R2_clean.fastq | head -n 5)


## Step 1b. Quality control with FastQC

Raw reads and trimmed reads were quality-checked using **FastQC**.  
- Initial QC was run immediately after receiving sequencing data.  
- A second QC run was performed after trimming (following Geneious Prime best practices). 

## Step 2a. Chromosome assembly with Unicycler


In [ ]:
!unicycler -1 R1.fastq.gz -2 R2.fastq.gz -o all_reads_unicycler


## Step 2b. Identify circularised contigs after Unicycler

Unicycler reports circularised contigs separately.  
- If contigs are already circular → assign as plasmids.  
- Remaining linear contigs are considered chromosome candidates and forwarded to Ragout.

## Step 3. Chromosome scaffolding with Ragout
Ragout requires Python 2.7, so we call it from the `ragout_env`.


In [ ]:
import subprocess

subprocess.run("""
conda run -n ragout_env ragout \
    -o chromosome_output \
    -s sibelia \
    chromosome_recipe.txt
""", shell=True, check=True)


## Step 4. Mapping reads back to chromosome assembly


In [ ]:
!bwa index chromosome_scaffolds.fasta
!bwa mem chromosome_scaffolds.fasta fixed_R1_clean.fastq fixed_R2_clean.fastq | samtools view -bS - > chrom_mapped.bam
!samtools sort -o chrom_mapped_sorted.bam chrom_mapped.bam
!samtools index chrom_mapped_sorted.bam


## Step 5. Extract unmapped reads and assemble plasmids with Unicycler


In [ ]:
!samtools view -b -f 12 -F 256 chrom_mapped_sorted.bam > unmapped.bam
!samtools fastq unmapped.bam -1 unmapped_R1.fastq -2 unmapped_R2.fastq -0 /dev/null -s /dev/null -n

!unicycler -1 unmapped_R1.fastq -2 unmapped_R2.fastq -o plasmid_unicycler


## Step 6. Iterative plasmid discovery and validation

Plasmid contigs assembled by Unicycler (unmapped reads) or RagTag scaffolding  
were searched against **PLSDB** using BLAST.  

- Hits ≥ 90% identity and ≥ 500 bp alignment were considered significant.  

In [ ]:
!blastn -query plasmid_unicycler/assembly.fasta -db plsdb -out plasmid_plsdb_blast.txt \
  -outfmt "6 qseqid sseqid pident length evalue bitscore" -evalue 1e-20

!awk '$3>=90 && $4>=500' plasmid_plsdb_blast.txt > plasmid_strong_hits.txt


## Step 7. RagTag plasmid scaffolding

- **Unicycler plasmids**: Circular contigs produced directly by Unicycler were retained as plasmids, regardless of size, since they represent complete replicons.  

- **RagTag scaffolds**: Contigs <10 kb were excluded from plasmid consideration.  
    - Short reference-guided scaffolds may represent incomplete fragments, repeats, or assembly artefacts rather than true plasmids.  
    - A 10 kb threshold ensures that only biologically meaningful plasmids are reported, while minimising false positives from RagTag assemblies.



In [ ]:
!ragtag.py scaffold reference.fasta plasmid_unicycler/assembly.fasta -o plasmid_ragtag


## Step 8. Circularisation of chromosome and plasmids

Both plasmid and chromosome assemblies were circularised using Circlator 
to standardise start positions and confirm closed molecules.

In [ ]:
# Circularise plasmid assemblies
!circlator fixstart plasmid_ragtag/ragtag.scaffold.fasta plasmid_circularised

# Circularise chromosome assembly
!circlator fixstart chromosome_output/scaffolds.fasta chromosome_circularised

## Step 9. Coverage analysis


In [ ]:
!bwa index chromosome.fasta
!bwa mem -t 8 chromosome.fasta fixed_R1_clean.fastq fixed_R2_clean.fastq | samtools sort -@ 4 -o chromosome.bam
!samtools index chromosome.bam

!bwa index plasmid.fasta
!bwa mem -t 8 plasmid.fasta fixed_R1_clean.fastq fixed_R2_clean.fastq | samtools sort -@ 4 -o plasmid.bam
!samtools index plasmid.bam

!samtools depth -a chromosome.bam | awk '{sum[$1]+=$3; count[$1]++} END {for (id in sum) print id, sum[id]/count[id]}' > chromosome_depth.txt
!samtools depth -a plasmid.bam | awk '{sum[$1]+=$3; count[$1]++} END {for (id in sum) print id, sum[id]/count[id]}' > plasmid_depth.txt


## Step 10. Assembly evaluation with QUAST and BUSCO


We used QUAST to evaluate both the initial Unicycler assembly (to capture contig statistics before scaffolding) and the final chromosome assembly (after Ragout).  
BUSCO was used on both assemblies to assess completeness at each stage.


In [ ]:
# QUAST: run on both initial and final assemblies
quast.py unicycler_assembly.fasta -o quast_unicycler_output
quast.py chromosome.fasta -o quast_chromosome_output

# BUSCO: run on both assemblies
busco -i unicycler_assembly.fasta -m genome -l bacteria -o busco_unicycler
busco -i chromosome.fasta -m genome -l bacteria -o busco_chromosome


## Step 11. Contamination check with CheckM2
**Note:** Genome assemblies were additionally screened for contamination using **CheckM2** (run via Galaxy platform).  
- This step was performed outside the conda/Jupyter environment.
- All reported genomes passed CheckM2 with acceptable contamination thresholds (<1%).  

## Special Case
**Note on BCC1689:**

For this strain, plasmid copy number was extremely low. A plasmid band was independently confirmed on agarose gel electrophoresis.  
To recover plasmid sequences:

- Plasmid search was performed **directly on raw reads before Unicycler**, to enrich plasmid-derived reads.  
- The enriched plasmid reads were assembled separately.  
- Chromosomal reads were then assembled with the standard pipeline (Unicycler → Ragout).  

This adjustment was necessary to account for low plasmid abundance, but did not affect overall genome completeness (BUSCO completeness = **98.3%**).


# Summary

- **Circularised chromosome**: `chromosome_circularised`
- **Circularised plasmids**: `plasmid_circularised`  
  - Includes all circular Unicycler plasmids (regardless of size)  
  - RagTag scaffolds <10 kb excluded as likely artefacts
- **Iterative plasmid search** continued until:
  - No new BLAST/PLSDB hits found, OR
  - New assemblies < 10 kb (excluded as unlikely plasmids; cutoff chosen because plasmids below this size are typically unstable or non-functional in bacteria)

- **Assembly evaluation**:
  - *Pre-scaffolding (Unicycler)*:  
    - Contig count, N50, and other statistics from `quast_unicycler_output`  
    - Completeness from `busco_unicycler`  
  - *Post-scaffolding (final chromosome)*:  
    - Genome size and gap statistics from `quast_chromosome_output`  
    - Completeness from `busco_chromosome`  

- **Coverage**: `chromosome_depth.txt`, `plasmid_depth.txt`